## MLflow's Model Registry

In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [3]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [4]:
client.create_experiment(name="my-cool-experiment")

'2'

Let's check the latest versions for the experiment with id `1`...

In [5]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [6]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 770bceee732d40609cc29817592731af, rmse: 6.2872
run id: 8f85311bbccc449a9e37d32ff386ba3a, rmse: 6.3024
run id: 315fe9fe76ae423a85a77f4cf95cc20a, rmse: 6.3062
run id: 338dd4f9b9304d4f8f8a6e470f7bcbbf, rmse: 6.3130
run id: c18b8022fcb84a5d89291ed7da05499a, rmse: 6.3242


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [10]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [22]:
run_id = "8f85311bbccc449a9e37d32ff386ba3a"

In [11]:
run_id = "338dd4f9b9304d4f8f8a6e470f7bcbbf"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

2023/04/21 12:28:31 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/04/21 12:28:31 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2023/04/21 12:28:31 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 3
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: creation_timestamp=1682058511086, current_stage='None', description=None, last_updated_timestamp=1682058511086, name='nyc-taxi-regressor', run_id='338dd4f9b9304d4f8f8a6e470f7bcbbf', run_link=None, source='/Users/rollan/Desktop/education/mlops-zoomcamp/02-experiment-tracking/mlruns/1/338dd4f9b9304d4f8f8a6e470f7bcbbf/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [12]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging
version: 2, stage: Archived
version: 3, stage: None


In [13]:
model_version = 3
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1682058511086, current_stage='Staging', description=None, last_updated_timestamp=1682060052394, name='nyc-taxi-regressor', run_id='338dd4f9b9304d4f8f8a6e470f7bcbbf', run_link=None, source='/Users/rollan/Desktop/education/mlops-zoomcamp/02-experiment-tracking/mlruns/1/338dd4f9b9304d4f8f8a6e470f7bcbbf/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [14]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1682058511086, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2023-04-21', last_updated_timestamp=1682060055329, name='nyc-taxi-regressor', run_id='338dd4f9b9304d4f8f8a6e470f7bcbbf', run_link=None, source='/Users/rollan/Desktop/education/mlops-zoomcamp/02-experiment-tracking/mlruns/1/338dd4f9b9304d4f8f8a6e470f7bcbbf/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [15]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [17]:
df = read_dataframe("../data/green_tripdata_2021-03.parquet")

In [23]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

/var/folders/6p/kwmkg6ps5d74czsbsp1p6zm80000gn/T/ipykernel_45333/2852634549.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')


'/Users/rollan/Desktop/education/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [24]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [25]:
X_test = preprocess(df, dv)

In [26]:
target = "duration"
y_test = df[target].values

In [34]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

[13:16:48] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 22.7 s, sys: 267 ms, total: 23 s
Wall time: 2.66 s


{'rmse': 6.248294328516825}

In [32]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

[13:16:03] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 21 s, sys: 196 ms, total: 21.2 s
Wall time: 2.24 s


{'rmse': 6.248294328516825}

In [33]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: creation_timestamp=1682054917665, current_stage='Production', description='', last_updated_timestamp=1682061385578, name='nyc-taxi-regressor', run_id='8f85311bbccc449a9e37d32ff386ba3a', run_link='', source='/Users/rollan/Desktop/education/mlops-zoomcamp/02-experiment-tracking/mlruns/1/8f85311bbccc449a9e37d32ff386ba3a/artifacts/models_mlflow', status='READY', status_message=None, tags={'model': 'xgboost'}, user_id=None, version=1>